# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from workflow_widgets import WorkflowWidget, WorkerPoolWidget, TailWidget
from droplet_workflow import droplet_wf
from example_workflow import example_wf
from aux_widgets import QueueWidget, NEWTAuthWidget, SSHAuthWidget, SSHTerminal, Space
import workflow_objects as kale

import ipywidgets as ipw
import time
from concurrent.futures import ThreadPoolExecutor
import traitlets
import networkx as nx
import IPython.display as disp

In [ ]:
wpw = WorkerPoolWidget()
wpw

In [ ]:
[task.index[droplet_wf] for task in droplet_wf.tag_dict['100A']]

In [ ]:
ww = WorkflowWidget(workflow=droplet_wf, worker_pool_widget=wpw)
ww

In [ ]:
def test_tag_to_selection():
    global s1, s2
    
    ww._tag_selector.index = 5 # parse
    ww._ta_tag_to_selection()
    ww._ta_append_all_children()
    ww._tag_selector.index = 12 # 100A
    ww._ta_selection_to_tag()
    s1 = ww.bqgraph.selected
    
    ww._ta_select_none()
    ww._ta_tag_to_selection()
    s2 = ww.bqgraph.selected
    
    d1 = set(s1).difference(s2)
    d2 = set(s2).difference(s1)
    
    if len(d1.union(d2)) == 0:
        print("Woohoo!")
    
    else:
        print("Fail :(")
        print("s1 - s2 = {}".format(sorted(list(d1))))
        print("s2 - s1 = {}".format(sorted(list(d2))))
        raise ValueError("Fail")
    

In [ ]:
test_tag_to_selection()

# Define simple workflow

In [ ]:
wfw = WorkflowWidget(example_wf, wpw)
wfw

In [ ]:
example_workflow = kale.Workflow(name='example')
for i in range(10):
    example_workflow.add_task(
        kale.CommandLineTask(
            name='echo_test_{num}',
            command='echo "Test #{num}" >> {out_file} &&',
            tags=['run'],
            output_files=["{out_file}"],
            params=dict(
                num=i,
                out_file="output.txt"
            )
        )
    )

In [ ]:
wfw.workflow.tag_dict

In [ ]:
wfw.workflow._new_tag('newtag')

In [ ]:
wfw._update_tag_selector()

# Connect to Cluster


In [ ]:
ssh = SSHAuthWidget(host='cori.nersc.gov', username='oevans')
ssh

In [ ]:
t = SSHTerminal(ssh, width=800)
t

# Create Worker Pool


# Visualize & Run Workflow

In [ ]:
exww.workflow.tag_dict

In [ ]:
selected = exww.bqgraph.selected
tasks = [exww.bqgraph.node_data[x]['name'] for x in selected]
[exww.workflow.get_task_by_name(name) for name in tasks]

In [ ]:
nx.dfs_preorder_nodes?

In [ ]:
exww.workflow.dag.o(node)

In [ ]:
exww.bqgraph.selected = list(range(5))

In [ ]:
example_wf.dag.nodes()[1].children

# Observe Output

In [ ]:
t = TailWidget('out.txt')
t

# Connect to NEWT

In [ ]:
a = NEWTAuthWidget()
a

In [ ]:
q = QueueWidget(a)
q